In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer

import cv2
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import pandas as pd
from scipy.sparse import csr_matrix

import random

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical

##Data Preprocessing

In [ ]:
dataset_path = os.listdir('/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700')

cell_types = os.listdir('/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700')
print (cell_types)

print('class=',len(cell_types))

['abnormal_carcinoma_in_situ', 'abnormal_light_dysplastic', 'abnormal_moderate_dysplastic', 'normal_columnar', 'abnormal_severe_dysplastic', 'normal_intermediate', 'normal_superficiel']
class= 7


In [ ]:
cells = []

for item in cell_types:
 # Get all the file names
 all_cell = os.listdir('/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700' + '/' +item)


 # Add them to the list
 for cellclass in all_cell:
    cells.append((item, str('/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700' + '/' +item) + '/' + cellclass))
    print(cells[:1])

Streaming output truncated to the last 5000 lines.
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_in_situ/153827595-153827657-001.BMP')]
[('abnormal_carcinoma_in_situ', '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/abnormal_carcinoma_i

In [ ]:
# Build a dataframe        
cell_df = pd.DataFrame(data=cells, columns=['class', 'image'])
print(cell_df.head())

                        class                                              image
0  abnormal_carcinoma_in_situ  /content/drive/MyDrive/UITM/SEM 3/Proposal/Aug...
1  abnormal_carcinoma_in_situ  /content/drive/MyDrive/UITM/SEM 3/Proposal/Aug...
2  abnormal_carcinoma_in_situ  /content/drive/MyDrive/UITM/SEM 3/Proposal/Aug...
3  abnormal_carcinoma_in_situ  /content/drive/MyDrive/UITM/SEM 3/Proposal/Aug...
4  abnormal_carcinoma_in_situ  /content/drive/MyDrive/UITM/SEM 3/Proposal/Aug...


In [ ]:
print("Total number of cell in the dataset: ", len(cell_df))
cell_count = cell_df['class'].value_counts()

print("cells in each class: ")
print(cell_count)

Total number of cell in the dataset:  5308
cells in each class: 
abnormal_carcinoma_in_situ      782
normal_superficiel              772
abnormal_moderate_dysplastic    762
abnormal_light_dysplastic       760
abnormal_severe_dysplastic      751
normal_columnar                 750
normal_intermediate             731
Name: class, dtype: int64


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
cell_count = cell_df['class'].value_counts()

In [ ]:
path = '/content/drive/MyDrive/UITM/SEM 3/Proposal/Aug700/'


im_size = 224

images = []
labels = []

for i in cell_types:
    data_path = path + str(i)  # entered in 1st folder and then 2nd folder and then 3rd folder
    filenames = [i for i in os.listdir(data_path)]
    #print(filenames)  # will get the names of all images
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)  # reading that image as array
        #print(img)  # will get the image as an array
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)
#labels   

In [ ]:
images = np.array(images)
images.shape

(5308, 224, 224, 3)

In [ ]:
images = images.astype('float32') / 255.0

In [ ]:
y=cell_df['class'].values
#print(y)

# for y
y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
#print (y)

y=y.reshape(-1,1)
onehotencoder = ColumnTransformer([('y', OneHotEncoder(), [0])])  #0 because class in column 0
Y= onehotencoder.fit_transform(y)
Y.shape

(5308, 7)

In [ ]:
type(cell_df)

pandas.core.frame.DataFrame

In [ ]:
#check dictionary
dict(zip(cell_df['class'].astype('category').cat.codes,cell_df['class'])) #add in report

{0: 'abnormal_carcinoma_in_situ',
 1: 'abnormal_light_dysplastic',
 2: 'abnormal_moderate_dysplastic',
 3: 'abnormal_severe_dysplastic',
 4: 'normal_columnar',
 5: 'normal_intermediate',
 6: 'normal_superficiel'}

##Split Dataset into Training & Testing

Data was splitted into 70% training and 30% testing

In [ ]:
images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.30, random_state=415)

#shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(3715, 224, 224, 3)
(3715, 7)
(1593, 224, 224, 3)
(1593, 7)


In [ ]:
print(type(train_y))
print(type(train_x))
print(type(test_y))
print(type(test_x))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [ ]:
#convert csr_matrix to numpy array
train_y=csr_matrix.toarray(train_y, order=None, out=None)
test_y=csr_matrix.toarray(test_y, order=None, out=None)

##Save to H5

In [ ]:
import h5py
import numpy as np

write

In [ ]:
hftrain = h5py.File('/content/drive/MyDrive/UITM/SEM 4/FYP/pickle file/700train.h5', 'w')
hftest = h5py.File('/content/drive/MyDrive/UITM/SEM 4/FYP/pickle file/700test.h5', 'w')

In [ ]:
hftrain.create_dataset('train_y', data=train_y)

<HDF5 dataset "train_y": shape (3715, 7), type "<f8">

In [ ]:
hftrain.create_dataset('train_x', data=train_x)

<HDF5 dataset "train_x": shape (3715, 224, 224, 3), type "<f4">

In [ ]:
hftest.create_dataset('test_x', data=test_x)

<HDF5 dataset "test_x": shape (1593, 224, 224, 3), type "<f4">

In [ ]:
hftest.create_dataset('test_y', data=test_y)

<HDF5 dataset "test_y": shape (1593, 7), type "<f8">

In [ ]:
hftrain.close()
hftest.close()